In [2]:
# kod: 1101010100
#      0101010011
# fitnes: ...
    
# kod: niska
# kod: [l1,l2,l3,...]
# kod: AST - stablo
    
# '''fact2 = 1
# for i in range(1, n):
#     if i % 2 == 0:
#         fact2 *= i
# return fact2'''

# '''
# if uslov:
#     f()
# else:
#     g()
# '''

# # resenje1 = rec|rec2|rec3
# # resenje2 = r.*
# # resenje3 = s
# # najkraci regex koji je tacan za sve iz grupe DA, a netacan za sve iz grupe NE
# # da:     ne:
# # rec     rec4
# # rec2    nerec
# # rec3    nerec2

# selekcija
# ukrstanje
# mutacija

In [3]:
import random

In [7]:
eval('(2+3)*(4-1)')

15

In [78]:
class Individual:
    
    def __init__(self, goal=100, num_operators=3, num_terminals=4):
#         self.code = [op1, op2, op3, a, b, c, d]
        self.code = []
        self.goal = goal
        self.num_operators = num_operators
        for i in range(num_operators):
            self.code.append(self.randomOperator())
        for i in range(num_terminals):
            self.code.append(self.randomTerminal())
            
        self.fitness = self.calcFitness()
        
    def __lt__(self, other):
        return self.fitness < other.fitness
        
    def __str__(self):
#         (a op2 b) op1 (c op3) d
        a = str(self.code[3])
        b = str(self.code[4])
        c = str(self.code[5])
        d = str(self.code[6])
        
        first = '(' + a + ' ' + self.code[1] + ' ' + b + ')'
        second = '(' + c + ' ' + self.code[2] + ' ' + d + ')'
        return first + ' ' + self.code[0] + ' ' + second
        
    def calcFitness(self):
        value = eval(str(self))
        # 15, 100 -> 85 -> 1/86, -85
        # 90, 100 -> 10 -> 1/11, -10
        # 100, 100 -> 0 -> 1/1,   0
        error = abs(self.goal - value)
        fitness = 1 / (1 + error)
        return fitness
    
    def randomOperator(self, forbidden={}):
        operators = {'+', '-', '*'}.difference(forbidden)
        return random.choice(list(operators))
    
    def randomTerminal(self, lb=-10, ub=10):
#         range(a,b) -> [a,b)
        return random.randrange(lb, ub+1)

In [79]:
i = Individual()
print(i)
i.fitness


(6 + 9) + (4 + -8)


0.011111111111111112

In [80]:
def selection(population):
    TOURNAMENT_SIZE = 5
    participants = random.sample(population, TOURNAMENT_SIZE)
    winner = max(participants)
    return winner

In [81]:
def getSubtree(root_index, subtree_indices, num_nodes):
    if root_index >= num_nodes:
        return
    subtree_indices.append(root_index)
    getSubtree(root_index * 2 + 1, subtree_indices, num_nodes)
    getSubtree(root_index * 2 + 2, subtree_indices, num_nodes)

In [82]:
def crossover(parent1, parent2, child1, child2):
#       0    1    2   3  4  5  6
#     [op1, op2, op3, a, b, c, d]
    root_index = random.randrange(1, len(parent1.code))
    subtree_indices = []
    getSubtree(root_index, subtree_indices, len(parent1.code))
    for i in range(len(parent1.code)):
        if i in subtree_indices:
            child1.code[i] = parent2.code[i]
            child2.code[i] = parent1.code[i]
        else:
            child1.code[i] = parent1.code[i]
            child2.code[i] = parent2.code[i]

In [83]:
def mutation(individual):
    MUTATION_PROB = 0.05
    for i in range(len(individual.code)):
        if random.random() < MUTATION_PROB:
            if i < individual.num_operators:
                individual.code[i] = individual.randomOperator(forbidden={individual.code[i]})
            else:
                individual.code[i] = individual.randomTerminal(lb=individual.code[i] - 10,
                                                               ub=individual.code[i] + 10)

In [90]:
POP_SIZE = 1000
NUM_GENERATIONS = 2000
ELITISM_SIZE = POP_SIZE // 3
if ELITISM_SIZE % 2 == 1:
    ELITISM_SIZE -= 1

population = [Individual(goal=1381) for _ in range(POP_SIZE)]
newPopulation = [Individual(goal=1381) for _ in range(POP_SIZE)]

for i in range(NUM_GENERATIONS):
    population.sort(reverse=True)
    if population[0].fitness == 1:
        break
    newPopulation[:ELITISM_SIZE] = population[:ELITISM_SIZE]
    for j in range(ELITISM_SIZE, POP_SIZE, 2):
        parent1 = selection(population)
        parent2 = selection(population)
        
        crossover(parent1, parent2, newPopulation[j], newPopulation[j+1])
        
        mutation(newPopulation[j])
        mutation(newPopulation[j+1])
        
        newPopulation[j].fitness = newPopulation[j].calcFitness()
        newPopulation[j+1].fitness = newPopulation[j+1].calcFitness()
        
    population = newPopulation
    
best = max(population)
print(f'solution: {best}, fitness: {best.fitness}')

solution: (9 * 9) * (8 + 9), fitness: 0.2


In [74]:
(-7 * 8) * (-8 - 10)

1008

In [86]:
(7 * 13) * (4 + 7)

1001

In [91]:
(9 * 9) * (8 + 9)

1377